# Fun With Types

We haven't used any of Julia's lovely [sophisticated type system](https://docs.julialang.org/en/latest/manual/types.html). Let's fix that now. I will show some code and there are undoubtedly *many* possible improvements, which we'll leave (for now) as an **exercise**. If you want, believe that I'm leaving things simple for clarity, rather than because I'm a simple person.

In [ ]:
abstract type AbstractHamiltonian{T, S:<AbstractMatrix} <: AbstractArray{T,N} end
# Hamiltonians may be real or complex

type SpinHamiltonian{T, S:<AbstractMatrix} <: AbstractHamiltonian
    L::Int
    basis::Vector # may be a vector of BitVectors/BitMatrices/Vectors/Matrices
    mat::S #this may be sparse, or Diagonal, or something else
end

ishermitian(Hamiltonian) = true

type TransverseFieldIsing{Tv, S} <: SpinHamiltonian{Tv, S}
    L::Int
    basis::Vector # may be a vector of BitVectors/BitMatrices/Vectors/Matrices
    mat::S #this may be sparse, or Diagonal, or something else
    h::Real
    function TransverseFieldIsing{Tv, S}(L::Integer, h::Real=0.) where Tv where S <: Matrix
        basis = generate_basis(L)
        H = zeros(2^L, 2^L)
        for (index, element) in enumerate(basis)
            # the diagonal part is easy
            diag_term = 0.
            for site in 1:L-1
                diag_term -= !xor(element[site], element[site+1])
            end
            H[index, index] = diag_term
            # off diagonal part
            for site in 1:L-1
                mask = falses(L)
                mask[site] = true
                new_element = xor.(element, mask)
                new_index = int_rep(new_element, L)
                H[index, new_index] = -h
            end
        end
        new(L, basis, Hermitian(H), h)
    end
    #function TransverseFieldIsing{Tv, S}(L::Integer, h::Real=0.) where Tv where S <: SparseMatrix
    #end
end

## Exercise

Write the constructor for the `TransverseFieldIsing` type that generates a *sparse* representation of the matrix. We'll need this to go to bigger system sizes. Some suggestions:

  1. Can we pre-allocate the arrays for the sparse matrix, since for this Hamiltonian we know *exactly* how many non-zero elements per row there could maximally be?
  2. Generating the basis separately is pretty slow - can we fully enumerate it using the Hamiltonian as we generate each row?
  3. It's probably a good idea to write some tests to make sure that our two representations match...

In [ ]:
# A little more fun with types
eigfact(A::TransverseFieldIsing; kwargs...) = eigfact(A.Mat; kwargs...)
eigvals(A::TransverseFieldIsing; kwargs...) = eigvals(A.Mat; kwargs...)
eigvecs(A::TransverseFieldIsing; kwargs...) = eigvecs(A.Mat; kwargs...)
eig(A::TransverseFieldIsing; kwargs...)     = eig(A.Mat; kwargs...)

## Exercise

We have many similar models in physics, including the quantum XY, XXZ, and Heisenberg models. Write a type and some constructors for each. Analogous to what we did for the Ising model, find the transition - does it look the same for different system sizes? You might be able to replicate some of your work by writing a few functions for "common" bond operations.

In [ ]:
type Heisenberg{Tv, S} <: SpinHamiltonian{Tv, S}
    L::Int
    basis::Vector
    mat::S
    #function Heisenberg{Tv, S}(L::Integer) where Tv where S <: Matrix
    #end
    #function Heisenberg{Tv, S}(L::Integer) where Tv where S <: SparseMatrix
    #end
end

Below, you'll find some more information about the models. If you're following along at home and not pressed for time, feel free to try all of them, but if you're doing this with me let's focus on the XXZ model:

$$ \hat{H} = -\sum_{\langle i, j \rangle} \hat{\sigma}_i^x \hat{\sigma}_j^x + \hat{\sigma}_i^y \hat{\sigma}_j^y +  \Delta\hat{\sigma}_i^z \hat{\sigma}_j^z $$

$\Delta$ is some real number between 0 and 1 that tunes the "anisotropy" of the system. To refresh our memories, the Pauli spin operators are:

In [1]:
σˣ = [0 1; 1 0]

2×2 Array{Int64,2}:
 0  1
 1  0

In [2]:
σʸ = [0 -im; im 0]

2×2 Array{Complex{Int64},2}:
 0+0im  0-1im
 0+1im  0+0im

In [3]:
σᶻ = [1 0; 0 -1]

2×2 Array{Int64,2}:
 1   0
 0  -1

If you're having trouble picturing how these *bond operators* (the pairs of $\sigma$s) should act, we can get Julia to help us:

In [5]:
kron(σˣ,σˣ)

4×4 Array{Int64,2}:
 0  0  0  1
 0  0  1  0
 0  1  0  0
 1  0  0  0

In [6]:
kron(σʸ,σʸ)

4×4 Array{Complex{Int64},2}:
  0+0im  0+0im  0+0im  -1+0im
  0+0im  0+0im  1+0im   0+0im
  0+0im  1+0im  0+0im   0+0im
 -1+0im  0+0im  0+0im   0+0im

We can actually make this much simpler to express in code using a little trick: rewrite in terms of the *raising and lowering operators* $\sigma^+$ and $\sigma^-$:

In [7]:
σ⁺ = (σˣ + im*σʸ)/2

2×2 Array{Complex{Float64},2}:
 0.0+0.0im  1.0+0.0im
 0.0+0.0im  0.0+0.0im

In [9]:
σ⁺*[0,1]

2-element Array{Complex{Float64},1}:
 1.0+0.0im
 0.0+0.0im

In [10]:
σ⁺*[1,0]

2-element Array{Complex{Float64},1}:
 0.0+0.0im
 0.0+0.0im

In [8]:
σ⁻ = (σˣ - im*σʸ)/2

2×2 Array{Complex{Float64},2}:
 0.0+0.0im  0.0+0.0im
 1.0+0.0im  0.0+0.0im

Which turns our Hamiltonian into (you can work this out by hand if you like):

$$ \hat{H}_{XXZ} = -\sum_{\langle i, j \rangle} 4\hat{\sigma}_i^+ \hat{\sigma}_j^- + 4\hat{\sigma}_i^- \hat{\sigma}_j^+ +  \Delta\hat{\sigma}_i^z \hat{\sigma}_j^z $$

Hopefully you can see why expressing it this way will be easier to implement with the bit-hacks. If you were doing everything with the matrix-vector ops this wouldn't matter but that doesn't scale super well. Now you can turn this into a function to make the XXZ model.